# lets play WORDLE GAME

In [1]:
from IPython.display import HTML, display
import requests
import random
import time

In [2]:
class ColorString:
    def __init__(self, text=""):
        self.parts = []
        if text:
            self.parts.append(f'<span>{text}</span> ')

    def add_text(self, text):
        self.parts.append(f'<span>{text}</span> ')

    def add_yellow(self, text):
        self.parts.append(f'<span style="color:gold;">{text}</span> ')

    def add_green(self, text):
        self.parts.append(f'<span style="color:green;">{text}</span> ')

    def __str__(self):
        """
        String olarak döndürülebilir. Renklendirme olmadan birleşmiş metin döner.
        """
        return "".join(part.strip('<span>').strip('</span>') for part in self.parts)

    def _repr_html_(self):
        """
        Jupyter Notebook'ta HTML olarak gösterim sağlar.
        """
        return "".join(self.parts)

def print_secret_word(guess,secretWord):
    printword=ColorString('')
    for letter in range(len(guess)):
        if guess[letter] not in secretWord:
            printword.add_text(guess[letter])
        elif guess[letter]!=secretWord[letter]:
            printword.add_yellow(guess[letter])
        else:
            printword.add_green(guess[letter])
    display(printword)



def download_file_from_drive(file_url, colab_path):
    if "drive.google.com" in file_url:
        file_id = file_url.split('/d/')[1].split('/')[0]
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    else:
        raise ValueError("The provided URL is not a valid Google Drive link.")

    response = requests.get(download_url, stream=True)

    if response.status_code == 200:
        with open(colab_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"File successfully downloaded to {colab_path}")
    else:
        raise Exception("Failed to download the file. Check the URL or file permissions.")

colab_path = "/content/words.txt"
file_url = "https://drive.google.com/file/d/1cjCZ-5uAHVFLXLV6HuSM98_QblWaXrBZ/view?usp=share_link"
download_file_from_drive(file_url,colab_path)



WORDLIST_FILENAME = "words.txt"

def load_words():

  inFile = open(WORDLIST_FILENAME, 'r')
  line = inFile.readline()
  wordlist = line.split()
  print("  ", len(wordlist), "words loaded.")
  return wordlist




def word_select():
  #You will use this function to generate a random word while solving the problem, this function returns a random word.
  return random.choice(word_list)


File successfully downloaded to /content/words.txt


In [3]:

word_list=load_words()
#Here is word_list, the list containing all English words, I will not delete it because you will use it in your problem. You can use it with this name whenever you need it.
egword=word_select()
print(egword)


   55900 words loaded.
horned


In [4]:

def isvalid(guess,secretWord):
    if guess not in word_list:
        return False
    if len(secretWord)!=len(guess):
      return False
    return True




def point_calculate(word,right):
    return len(word)*right



def guess_control(guess,word):
    if guess==word:
        return True
    else:
      return False


def print_length_of_secretWord(secretWord):
  print(f'Secret word have {len(secretWord)} letter')


In [5]:
def wordle():
    secretWord=word_select()
    print(secretWord)
    right=len(secretWord)
    print_length_of_secretWord(secretWord)
    while True:
        time.sleep(1)
        guess=input('Enter your guess:')
        if guess=='q':
          print(f'game over\nWord was: {secretWord}')
          break
        if isvalid(guess,secretWord):
          if guess_control(guess,secretWord):
            print(f'You found the word congrats!!!\nYour point is: {point_calculate(secretWord,right)}')
            break
          right-=1
          if right==0:
            print(f'Failed!!! You have no more rights\nWord was: {secretWord}')
            break
          print(f'You have {right} right to guess')
          print_secret_word(guess,secretWord)
          print('wordle')
          continue

        else:
          print(f'Please enter a valid word...\nYou have {right} right to guess')


In [6]:
wordle()

whipsaws
Secret word have 8 letter
Enter your guess:astronot
Please enter a valid word...
You have 8 right to guess
Enter your guess:dementia
You have 7 right to guess


wordle
Enter your guess:whipsees
Please enter a valid word...
You have 7 right to guess
Enter your guess:airfrier
Please enter a valid word...
You have 7 right to guess
Enter your guess:airlines
You have 6 right to guess


wordle
Enter your guess:whipsaws
You found the word congrats!!!
Your point is: 48


# Lets now build wordle solver

In [7]:
def word_select(wordlist):
    return random.choice(wordlist)

In [8]:
def updatelist1(guess, secretWord):
    matches = []  # Correct letters in correct positions
    i = 0         # Initialize index
    while i < len(secretWord):
        if guess[i] == secretWord[i]:  # if the letters match
            matches.append([guess[i], i])  # Save the letter and its position
        i += 1  # Move to the next index
    return matches

In [9]:
def updatelist2(guess, secretWord, list1):
    wrong_positions = []  # letters in wrong position
    used_spots = []      # spots we already used

    # Remember used spots from list1
    i = 0
    while i < len(list1):
        used_spots.append(list1[i][1])
        i += 1

    # Look at each letter in our guess
    i = 0
    while i < len(guess):
        # Skip if we already used this spot
        if i not in used_spots:
            letter = guess[i]
            # Look for this letter in secret word
            j = 0
            while j < len(secretWord):
                # If we find the letter and spot isn't used
                if letter == secretWord[j] and j not in used_spots:
                    # Remember letter and its wrong spot
                    wrong_positions.append([letter, i])
                    used_spots.append(j)
                    break
                j += 1
        i += 1

    return wrong_positions

In [10]:
def updatelist3(guess, secretWord, list1, list2):
    wrong_letters = []   # letters not in word
    right_letters = []   # letters we got right

    # Remember letters we got right
    i = 0
    while i < len(list1):
        right_letters.append(list1[i][0])
        i += 1

    i = 0
    while i < len(list2):
        right_letters.append(list2[i][0])
        i += 1

    # Check each letter in guess
    i = 0
    while i < len(guess):
        letter = guess[i]
        # If letter isn't in secret word and we haven't found it yet
        if letter not in secretWord and letter not in wrong_letters:
            wrong_letters.append(letter)
        # If letter is in word but we found too many of it
        elif letter not in right_letters and letter not in wrong_letters:
            wrong_letters.append(letter)
        i += 1

    return wrong_letters

In [12]:
def cleanwordlist(wordlist, secretWord):
    # Make a new empty list for words with same length
    cleaned_list = []
    for word in wordlist:
        if len(word) == len(secretWord):
            cleaned_list.append(word)
    return cleaned_list

In [13]:
def wordle(guess,secretWord):
  """
  Compare a guess with secretWord and return three lists:
    - list1: Correct letters in correct positions
    - list2: Correct letters in wrong positions
    - list3: Letters not in word
  """
  # Get feedback about the guess using our helper functions
  list1 = updatelist1(guess, secretWord)
  list2 = updatelist2(guess, secretWord, list1)
  list3 = updatelist3(guess, secretWord, list1, list2)

  return list1, list2, list3

In [14]:
def generate_guess(wordlist, list1, list2, list3, secretWord):
    possible_words = cleanwordlist(wordlist, secretWord)  # list of words with matching length
    newguess_words = []  # words that pass our checks

    # Check each possible word
    for word in possible_words:
        # Assume word is good until we prove it's not
        keeptheword = True

        # Check list1 (right letters, right spots)
        for item in list1:
            letter = item[0]    # Get the letter
            position = item[1]  # Get its position
            if word[position] != letter:
                keeptheword = False
                break

        # If word failed list1 check, try next word
        if not keeptheword:
            continue

        # Check list2 (right letters, wrong spots)
        for item in list2:
            letter = item[0]           # Get the letter
            wrong_position = item[1]   # Get position where it shouldn't be
            if letter not in word or word[wrong_position] == letter:
                keeptheword = False
                break

        # If word failed list2 check, try next word
        if not keeptheword:
            continue

        # Check list3 (wrong letters)
        for wrong_letter in list3:
            if wrong_letter in word:  # If this wrong letter appears in word
                keeptheword = False
                break

        # If word passed all checks, add it to newguess_words list
        if keeptheword:
            newguess_words.append(word)

    # Return a random newguess_word if we found any
    if newguess_words:
        return word_select(newguess_words)
    return None

In [15]:
def optimize_wordle():
    # Load words and pick secret word
    wordlist = load_words()
    secretWord = word_select(wordlist)

    print("\nSecretword is:", secretWord)
    print("\nComputer will try to find this word...")

    # To Keep track of guesses
    attempts = 0
    guessed_words = []

    # Store what we learn from each guess
    correct = []     # right letter, right spot / use of list1
    misplaced = []   # right letter, wrong spot / use of list2
    wrong = []       # wrong letters / use of list3

    while True:
        # For first guess, just pick any random word with same length
        if attempts == 0:
            guess = word_select(cleanwordlist(wordlist, secretWord))
        # For later guesses, use what we learned so far
        else:
            guess = generate_guess(wordlist, correct, misplaced, wrong, secretWord)

        attempts += 1
        guessed_words.append(guess)

        print("\nComputer's guess is:", guess)

        # If we found the word, we're done
        if guess == secretWord:
            print("\nThe computer guessed the word in", attempts, "attempts.")
            print("List of computer's guesses:", guessed_words)
            return attempts

        else:
            # Get feedback about our guess
            correct, misplaced, wrong = wordle(guess, secretWord)

# Run Main Function
optimize_wordle()

   55900 words loaded.

Secretword is: jimmy

Computer will try to find this word...

Computer's guess is: pomps

Computer's guess is: nimbi

Computer's guess is: pimps

Computer's guess is: aimed

Computer's guess is: nimbi

Computer's guess is: rimed

Computer's guess is: limns

Computer's guess is: timer

Computer's guess is: pimas

Computer's guess is: mimed

Computer's guess is: limps

Computer's guess is: mimic

Computer's guess is: gimpy

Computer's guess is: jimmy

The computer guessed the word in 14 attempts.
List of computer's guesses: ['pomps', 'nimbi', 'pimps', 'aimed', 'nimbi', 'rimed', 'limns', 'timer', 'pimas', 'mimed', 'limps', 'mimic', 'gimpy', 'jimmy']


14